In [ ]:
import numpy as np
import re

from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import math
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import pandas as pd
pd.set_option('max_columns', None)


import warnings
warnings.filterwarnings("ignore")



In [ ]:
df = pd.read_json("reviews_Automotive_5.json.gz", lines=True)
df.head()

In [ ]:
df['positive'] = df.overall.apply(lambda x: 1 if x >= 4 else 0)
print(df.shape)
df.head()

# Data Preprocessing

In [ ]:
def remove_tags(string):
    removelist = ""
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    result = re.sub(r'\W+', ' ',result)    #remove non-alphanumeric characters 
    result = result.lower()
    return result

review_df = df[['reviewText','overall']]
review_df['reviewText']=review_df['reviewText'].apply(lambda cw : remove_tags(cw))

review_df

### remove stopwords from the review. Stopwords are commonly used words like ‘and’, ‘the’, ‘at’ that do not add any special meaning or significance to a sentence

In [ ]:
review_df['reviewText'] = review_df['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
review_df['reviewText']

In [ ]:

review_list = [i for i in review_df['reviewText']][:1000]
review_list[:2]

In [ ]:
t=Tokenizer()
t.fit_on_texts(review_list)
sequences =t.texts_to_sequences(review_list)

In [ ]:
print('sequences : ',sequences[:2],'\n')


N = len([item for sublist in sequences[:2] for item in sublist])

print('word_index : ',{k: t.word_index[k] for k in list(t.word_index)[:N]})


In [ ]:
len_mat=[]
for i in range(len(sequences)):
    len_mat.append(len(sequences[i]))

In [ ]:
len(len_mat)

In [4]:
%%writefile data_frame.py

import pandas as pd
pd.set_option('max_columns', None)



def eda_nltk():
    
    if dataframe.split('.')[-1] == 'json' or dataframe.split('.')[-1] == 'csv' or dataframe.split('.')[-1] == 'xlsx':

        if dataframe.split('.')[-1] == 'json':
            df = pd.read_json(dataframe, lines=True)
#             print(df.head())
#             print('*'*60)
        elif dataframe.split('.')[-1] == 'csv':
            df = pd.read_csv(dataframe)
#             print(df.head())
#             print('*'*60)
        elif dataframe.split('.')[-1] == 'xlsx':
            df = pd.read_excel(dataframe)
#             print(df.head())
#             print('*'*60)
        else:
            print('Check your file name')
        return df
    
    else:
       
        if dataframe.split('.')[-2]  == 'json':
            df = pd.read_json(dataframe, lines=True)
#             print(df.head())
#             print('*'*60)
        elif dataframe.split('.')[-2]  == 'csv':
            df = pd.read_csv(dataframe)
#             print(df.head())
#             print('*'*60)
        elif dataframe.split('.')[-2]  == 'xlsx':
            df = pd.read_excel(dataframe)
#             print(df.head())
#             print('*'*60)
        else:
            print('Check your file name')
    
        return df
    


Overwriting data_frame.py


In [ ]:
%%writefile data_frame2.py

import pandas as pd
pd.set_option('max_columns', None)

# dataframe = 'reviews_Automotive_5.json.gz'
# print(dataframe)

def eda_nltk(dataframe = input('Input Dataframe: ')):
    if dataframe.split('.')[-1] == 'csv' or dataframe.split('.')[-1] == 'json':
        if dataframe.split('.')[-1] == 'csv':
            df = pd.read_csv(dataframe)
        elif dataframe.split('.')[-1] == 'json':
            df = pd.read_json(dataframe, lines=True)
        else:
            print('Check your file name')
        return df
    else:
        if dataframe.split('.')[-2] == 'csv':
            df = pd.read_csv(dataframe)
        elif dataframe.split('.')[-2] == 'json':
            df = pd.read_json(dataframe, lines=True)
        else:
            print('Check your file name')
        return df
    
# eda_nltk()

In [3]:
from data_frame2 import eda_nltk

dataframe = input(': ')
eda_nltk(dataframe)

: reviews_Automotive_5.json.gz


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"
...,...,...,...,...,...,...,...,...,...
20468,A1KB43BI085A48,B00KIAQ8VW,Mike F.,"[1, 1]",I've bought 10 of these over the past year to ...,5,Excellent quality for the price,1396915200,"04 8, 2014"
20469,A3HIUWKL4D853W,B00KIAQ8VW,RayH,"[0, 0]","OK, I have to admit, the price of this item di...",2,Light weight neck and face cover with whimsica...,1391472000,"02 4, 2014"
20470,A1S95WASXJ02VX,B00KIAQ8VW,Roberto Aguilar Jr.,"[0, 0]",I love my skull face mask. It makes me outstan...,5,I love to ride with my Skull Face Mask.,1386547200,"12 9, 2013"
20471,A2IID0Z6EHF5KP,B00KIAQ8VW,Soarer,"[0, 0]",This mask of course as described is half-face ...,5,Great neck protection,1357171200,"01 3, 2013"


In [ ]:
df
